In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

import tensorflow as tf
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow import squeeze

import numpy as np

print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

import configparser
config = configparser.ConfigParser()
config.read('config.ini')

import wandb
from wandb.keras import WandbCallback
wandb.login()

run = wandb.init(project="basic-nn-tensorflow", id="lr-scheduler-adam-bce")
wandb.config.update(config)



2.9.2
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mato-katkovcin (nsiete-hrnciar-katkovcin). Use `wandb login --relogin` to force relogin


Initial load of config file, where we have hyperparameters we are training our model

In [2]:
start_lr = config['tensorflow'].getfloat('start_lr')
max_lr = config['tensorflow'].getfloat('max_lr')
epochs = config['default'].getint('epochs')
steps_per_epoch = config['tensorflow'].getint('steps_per_epoch')
anneal_strategy = config['tensorflow']['strategy']

start_lr, max_lr, epochs, steps_per_epoch, anneal_strategy

(0.001, 0.01, 500, 1, 'cos')

## Data loading and train-test split

Initially, we imported the processed data and formed two collections of information: predictors (X) and labels (y). We then divided these sets into two parts, one for training (80%) and another for testing (20%). To maintain consistency in the split, we also specified a random state seed.

In [3]:
df = pd.read_csv('Data/cleaned.csv', index_col=0)
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0.352941,0.670968,0.489796,0.304348,0.186899,0.314928,0.234415,0.483333,1
1,0.058824,0.264516,0.428571,0.239130,0.106370,0.171779,0.116567,0.166667,0
2,0.470588,0.896774,0.408163,0.271739,0.186899,0.104294,0.253629,0.183333,1
3,0.058824,0.290323,0.428571,0.173913,0.096154,0.202454,0.038002,0.000000,0
4,0.000000,0.600000,0.163265,0.304348,0.185096,0.509202,0.943638,0.200000,1
...,...,...,...,...,...,...,...,...,...
763,0.588235,0.367742,0.530612,0.445652,0.199519,0.300613,0.039710,0.700000,0
764,0.117647,0.503226,0.469388,0.217391,0.106370,0.380368,0.111870,0.100000,0
765,0.294118,0.496774,0.489796,0.173913,0.117788,0.163599,0.071307,0.150000,0
766,0.058824,0.529032,0.367347,0.271739,0.186899,0.243354,0.115713,0.433333,1


In [4]:
X, y = df.drop('Outcome', axis=1).values, df.Outcome.values

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle=True, random_state=42)

X_train = tf.cast(tf.constant(X_train), tf.float32)
X_test = tf.cast(tf.constant(X_test), tf.float32)
y_train = tf.cast(tf.expand_dims(y_train, axis=1), tf.float32)
y_test = tf.cast(tf.expand_dims(y_test, axis=1), tf.float32)

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-03-29 00:15:54.825560: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-29 00:15:54.825703: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Our neural network is composed of three layers: an input layer, a hidden layer, and an output layer. The activation function used in all layers except the output layer is ReLU, while the output layer uses the Sigmoid activation function, which ensures the output is within the range of 0 to 1. To perform binary classification, we utilized binary cross entropy loss (BCELoss) and Adam optimizer.

Furthermore, we incorporated a learning rate scheduler to adjust the learning rate, making it higher at the start of training and gradually decreasing as we approach the minimum.

In [5]:
def schedule(epoch):
    cycle = 0
    if epoch < (epochs // 2):
        cycle = 1 + (epoch * steps_per_epoch) // (epochs // 2)
        factor = float(cycle * max_lr) / float(epochs)
    else:
        cycle = 1 + ((epochs - epoch) * steps_per_epoch) // (epochs // 2)
        factor = float(cycle * max_lr) / float(epochs)
        factor = max_lr - factor

    if anneal_strategy == 'cos':
        lr = factor * (1 + tf.math.cos(tf.constant(np.pi) * (epoch % (epochs // 2)) / (epochs // 2))) / 2
    elif anneal_strategy == 'linear':
        lr = factor * (epochs - epoch) / epochs
    else:
        raise ValueError('Invalid anneal strategy')

    return lr

In [6]:
def NeuralNetworkModel():
    model = tf.keras.Sequential([
    tf.keras.layers.Dense(8, activation='relu', input_shape=[len(df.keys()) - 1]),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])
    return model

model = NeuralNetworkModel()

loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=start_lr)
lr_scheduler = LearningRateScheduler(schedule=schedule)

model.compile(loss=loss_fn, optimizer=optimizer, metrics=['accuracy'])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 4)                 36        
                                                                 
 dense_2 (Dense)             (None, 1)                 5         
                                                                 
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________


During the training process, we measure the loss and accuracy while the scheduler adjusts the learning rate. When testing, we measure the testing loss,  and accuracy and all these metrics are recorded and stored in wandb.

In [8]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=config['default'].getint('epochs'), verbose=2, callbacks=[WandbCallback(valdiation_data=(X_test, y_test), log_weights=True)])

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/500


2023-03-29 00:15:55.660598: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
/opt/homebrew/Caskroom/miniconda/base/envs/nsietetensor/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)
2023-03-29 00:15:55.912703: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-29 00:15:56.381286: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


INFO:tensorflow:Assets written to: /Users/martinkatkovcin/Documents/School/NSIETE/Zadanie1/wandb/run-20230329_001550-lr-scheduler-adam-bce/files/model-best/assets


wandb: Adding directory to artifact (/Users/martinkatkovcin/Documents/School/NSIETE/Zadanie1/wandb/run-20230329_001550-lr-scheduler-adam-bce/files/model-best)... Done. 0.0s


...

20/20 - 1s - loss: 0.3357 - accuracy: 0.8616 - val_loss: 0.4070 - val_accuracy: 0.8506 - 657ms/epoch - 33ms/step
Epoch 259/500
20/20 - 0s - loss: 0.3340 - accuracy: 0.8616 - val_loss: 0.4114 - val_accuracy: 0.8506 - 182ms/epoch - 9ms/step
Epoch 260/500
20/20 - 0s - loss: 0.3344 - accuracy: 0.8616 - val_loss: 0.4096 - val_accuracy: 0.8506 - 224ms/epoch - 11ms/step
Epoch 261/500
20/20 - 0s - loss: 0.3348 - accuracy: 0.8616 - val_loss: 0.4121 - val_accuracy: 0.8442 - 183ms/epoch - 9ms/step
Epoch 262/500
20/20 - 0s - loss: 0.3341 - accuracy: 0.8599 - val_loss: 0.4131 - val_accuracy: 0.8442 - 180ms/epoch - 9ms/step
Epoch 263/500
20/20 - 0s - loss: 0.3356 - accuracy: 0.8632 - val_loss: 0.4113 - val_accuracy: 0.8506 - 180ms/epoch - 9ms/step
Epoch 264/500
20/20 - 0s - loss: 0.3341 - accuracy: 0.8648 - val_loss: 0.4109 - val_accuracy: 0.8442 - 181ms/epoch - 9ms/step
Epoch 265/500
20/20 - 0s - loss: 0.3325 - accuracy: 0.8648 - val_loss: 0.4087 - val_accuracy: 0.8506 - 174ms/epoch - 9ms/step
Epoc

wandb: Adding directory to artifact (/Users/martinkatkovcin/Documents/School/NSIETE/Zadanie1/wandb/run-20230329_001550-lr-scheduler-adam-bce/files/model-best)... Done. 0.0s


20/20 - 1s - loss: 0.3342 - accuracy: 0.8632 - val_loss: 0.4060 - val_accuracy: 0.8506 - 655ms/epoch - 33ms/step
Epoch 268/500
20/20 - 0s - loss: 0.3325 - accuracy: 0.8616 - val_loss: 0.4081 - val_accuracy: 0.8442 - 174ms/epoch - 9ms/step
Epoch 269/500
20/20 - 0s - loss: 0.3315 - accuracy: 0.8648 - val_loss: 0.4073 - val_accuracy: 0.8506 - 180ms/epoch - 9ms/step
Epoch 270/500
20/20 - 0s - loss: 0.3312 - accuracy: 0.8681 - val_loss: 0.4076 - val_accuracy: 0.8506 - 174ms/epoch - 9ms/step
Epoch 271/500
20/20 - 0s - loss: 0.3308 - accuracy: 0.8681 - val_loss: 0.4098 - val_accuracy: 0.8442 - 172ms/epoch - 9ms/step
Epoch 272/500
20/20 - 0s - loss: 0.3301 - accuracy: 0.8632 - val_loss: 0.4134 - val_accuracy: 0.8442 - 220ms/epoch - 11ms/step
Epoch 273/500
20/20 - 0s - loss: 0.3304 - accuracy: 0.8648 - val_loss: 0.4117 - val_accuracy: 0.8442 - 175ms/epoch - 9ms/step
Epoch 274/500
20/20 - 0s - loss: 0.3303 - accuracy: 0.8648 - val_loss: 0.4121 - val_accuracy: 0.8506 - 188ms/epoch - 9ms/step
Epoc

wandb: Adding directory to artifact (/Users/martinkatkovcin/Documents/School/NSIETE/Zadanie1/wandb/run-20230329_001550-lr-scheduler-adam-bce/files/model-best)... Done. 0.0s


20/20 - 1s - loss: 0.3280 - accuracy: 0.8648 - val_loss: 0.4048 - val_accuracy: 0.8506 - 652ms/epoch - 33ms/step
Epoch 283/500
20/20 - 0s - loss: 0.3277 - accuracy: 0.8681 - val_loss: 0.4071 - val_accuracy: 0.8506 - 163ms/epoch - 8ms/step
Epoch 284/500
20/20 - 0s - loss: 0.3270 - accuracy: 0.8648 - val_loss: 0.4112 - val_accuracy: 0.8506 - 164ms/epoch - 8ms/step
Epoch 285/500
20/20 - 0s - loss: 0.3298 - accuracy: 0.8632 - val_loss: 0.4131 - val_accuracy: 0.8506 - 161ms/epoch - 8ms/step
Epoch 286/500
20/20 - 0s - loss: 0.3269 - accuracy: 0.8664 - val_loss: 0.4088 - val_accuracy: 0.8506 - 201ms/epoch - 10ms/step
Epoch 287/500
20/20 - 0s - loss: 0.3265 - accuracy: 0.8664 - val_loss: 0.4105 - val_accuracy: 0.8571 - 165ms/epoch - 8ms/step
Epoch 288/500
20/20 - 0s - loss: 0.3272 - accuracy: 0.8681 - val_loss: 0.4097 - val_accuracy: 0.8506 - 169ms/epoch - 8ms/step
Epoch 289/500
20/20 - 0s - loss: 0.3267 - accuracy: 0.8664 - val_loss: 0.4091 - val_accuracy: 0.8506 - 165ms/epoch - 8ms/step
Epoc

In [9]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

,loss,accuracy,val_loss,val_accuracy,epoch
495,0.300755,0.879479,0.428386,0.857143,495
496,0.300480,0.881108,0.430269,0.850649,496
497,0.301427,0.881108,0.430069,0.857143,497
498,0.300552,0.884365,0.431169,0.850649,498
499,0.300718,0.882736,0.434486,0.850649,499


At the end, we calculate training and validation accuracy.

In [10]:
acc = (hist['accuracy'].tail().sum())*100/5 
val_acc = (hist['val_accuracy'].tail().sum())*100/5 

print("Training Accuracy = {}% and Validation Accuracy = {}%".format(round(acc, 2), round(val_acc, 2)))

Training Accuracy = 88.18% and Validation Accuracy = 85.32%


In [11]:
tf.keras.models.save_model(model, "models/model_tf.pth")
wandb.save('runs/pima_run_2023-03-28-tf-adam-bce')
wandb.finish()

INFO:tensorflow:Assets written to: models/model_tf.pth/assets


accuracy,▁▁▁▁▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇██████████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▆▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▁▁▁▅▅▆▆▆▆▇▆▇▇▇█████████████████▇▇██████
val_loss,█▇▅▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▂▂
accuracy,0.88274
best_epoch,281
best_val_loss,0.40483
epoch,499
loss,0.30072
val_accuracy,0.85065
